## Imports and Setup
Sets up the environment and defines the path to custom modules.

In [ ]:
import sagemaker
import boto3
import json
import os
import sys
from sagemaker.tuner import HyperparameterTuner
from sagemaker.serverless import ServerlessInferenceConfig

# Session Configuration
Sagemaker session configuration

In [ ]:
import sagemaker
import boto3

# --- 1. Infrastructure Connection (SSM) ---
region = boto3.Session().region_name

# Project Configuration (Must match your terraform.tfvars)
project_name = "cbis-ddsm"
env = "dev"


# Function to read Parameter Store
def get_ssm_parameter(name):
    try:
        ssm = boto3.client('ssm', region_name=region)
        response = ssm.get_parameter(Name=name)
        return response['Parameter']['Value']
    except Exception as e:
        print(f"⚠️ Error reading parameter {name}: {e}")
        return None


print("🔄 Loading infrastructure configuration via SSM...")

# Load Dynamic Variables
bucket_ssm = get_ssm_parameter(f"/{project_name}/{env}/s3_bucket_name")
role_ssm = get_ssm_parameter(f"/{project_name}/{env}/sagemaker_role_arn")

# --- 2. Session Initialization ---
if bucket_ssm and role_ssm:
    bucket = bucket_ssm
    role = role_ssm

    # IMPORTANT: Initialize Session with the Terraform bucket
    # This forces SageMaker to use your specific bucket for all artifacts
    sess = sagemaker.Session(default_bucket=bucket)

    print("✅ MLOps Mode Activated: Using Terraform resources.")
else:
    # Fallback to standard mode (Sandbox)
    sess = sagemaker.Session()
    bucket = sess.default_bucket()
    role = sagemaker.get_execution_role()
    print("⚠️ Sandbox Mode: Using default resources.")

prefix = "cbis-ddsm-classification"

print(f"Region: {region}")
print(f"Bucket: {bucket}")
print(f"Role:   {role.split('/')[-1]}")

## Retrieve and Deploy the Best Model
Auto-Discover Latest Tuning Job and Deploy Serveless Inference Endpoint

In [ ]:
# Auto-Discover Latest Tuning Job
print("\n🔎 Searching for the latest completed Hyperparameter Tuning Job...")

sm_client = boto3.client('sagemaker')

# List jobs sorted by creation time (descending)
response = sm_client.list_hyperparameter_tuning_jobs(
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=1,
    StatusEquals='Completed' # Only looks for successful jobs
)

if response['HyperparameterTuningJobSummaries']:
    # Grab the name of the most recent one
    tuning_job_name = response['HyperparameterTuningJobSummaries'][0]['HyperparameterTuningJobName']
    print(f"✅ Latest Tuning Job found: {tuning_job_name}")
else:
    # STOP EXECUTION HERE IF NOT FOUND
    error_msg = "No completed Tuning Job found! Please run notebook 02 first."
    print(error_msg)
    raise ValueError(error_msg) # This stops the notebook

# Retrieve the Best Model
print(f"\nAttaching to job '{tuning_job_name}' to find the best model...")
tuner = HyperparameterTuner.attach(tuning_job_name)

# Find out which training job was the best
best_training_job = tuner.best_training_job()
print(f"✅ The winning model was training job: {best_training_job}")

# Serverless Endpoint Deployment
print("\nConfiguring Serverless Inference to save costs...")

# Memory Configuration: 4GB is safe for ResNet50
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=4096,
    max_concurrency=5
)

print("Starting deployment... This might take a few minutes.")

# Deploy the best model
predictor = tuner.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name="cbis-ddsm-serverless-endpoint"
)

print(f"🚀 Serverless Endpoint ready! Name: {predictor.endpoint_name}")

# Inference Test
Invoke the Endpoint to test the Inference

In [ ]:

# Prepare a Test Image ---
s3 = boto3.client('s3')
print("\nSelecting a random image from validation set for testing...")

# Download validation manifest
s3.download_file(bucket, f"{prefix}/metadata/validation.lst", "val_temp.lst")

# Read the first line to get a valid image
with open("val_temp.lst", "r") as f:
    line = f.readline()
    parts = line.strip().split('\t')
    test_label = parts[1]
    test_image_path = parts[2]

# Remove temp file
os.remove("val_temp.lst")

print(f"Testing with image: {test_image_path}")
print(f"Real Label (Expected): {test_label} (0=Benign, 1=Malignant)")

# Download the actual image file
image_name = "test_image.jpg"
s3.download_file(bucket, f"{prefix}/images/{test_image_path}", image_name)

# Inference
print("Sending image to Serverless Endpoint...")

with open(image_name, 'rb') as f:
    payload = f.read()

# Invoke the endpoint
try:
    response = predictor.predict(payload, initial_args={'ContentType': 'application/x-image'})

    # Result Interpretation
    result = json.loads(response)
    prob_benign = result[0]
    prob_malignant = result[1]

    print("\n--- AI Diagnostic Result ---")
    print(f"Cancer Probability:   {prob_malignant:.4f} ({prob_malignant*100:.2f}%)")
    print(f"Benign Probability:   {prob_benign:.4f} ({prob_benign*100:.2f}%)")

    if prob_malignant > 0.5:
        print("Diagnosis: POSITIVE (Malignant)")
    else:
        print("Diagnosis: NEGATIVE (Benign)")

except Exception as e:
    print(f"Error during inference: {str(e)}")

## Cleanup ( Optional )
Used to delete endpoint after testing ( uncomment for default )

In [ ]:
# Cleanup ---
# Uncomment below to delete endpoint after testing
# print("\nDeleting endpoint...")
# predictor.delete_endpoint()
# print("Endpoint deleted.")